In [6]:
# Install required libraries
!pip install datasets transformers torch scikit-learn

# Import necessary libraries
import pandas as pd
import numpy as np
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from sklearn.model_selection import train_test_split

# Step 1: Load the Dataset
# Load the Hugging Face dataset
dataset = load_dataset("SKNahin/bengali-transliteration-data")

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['bn', 'rm'],
        num_rows: 5006
    })
})

In [32]:
# Check the column names in the dataset
print(dataset_dict["train"].column_names)


['bn', 'rm']


In [33]:
# Split the dataset into training and validation subsets (80/20 split)
dataset_dict = dataset['train'].train_test_split(test_size=0.2, seed=42)

In [34]:
# Step 2: Data Preprocessing
# Load a tokenizer suitable for sequence-to-sequence tasks
tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50")

In [35]:
# Inspect dataset rows for None or empty values
invalid_rows = dataset_dict.filter(lambda example: not example['rm'] or not example['bn'])
print(invalid_rows)


Filter:   0%|          | 0/4004 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1002 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['bn', 'rm'],
        num_rows: 0
    })
    test: Dataset({
        features: ['bn', 'rm'],
        num_rows: 0
    })
})


In [36]:
# Validate dataset structure
print(dataset_dict)

# Check for issues in the first few examples
print(dataset_dict["train"][0:5])  # Inspect the first 5 rows


DatasetDict({
    train: Dataset({
        features: ['bn', 'rm'],
        num_rows: 4004
    })
    test: Dataset({
        features: ['bn', 'rm'],
        num_rows: 1002
    })
})
{'bn': ['এটা কোনো পোস্ট হলো মিয়া আবাল', 'ডিলিট করেন পোস্ট…', 'জি ভাই অসাধারণ হইছে বাট ফুল ডিটেইলস জানতে পারলাম না, আর এটা কি বিডি তে সব জায়গাতেই পাওয়া যাবে?', 'ভাই...আর ২৪ আওয়ার ওয়েট করেন..আমার এফবি আইডি ব্যাক আসবে আমি রিকুয়েস্ট পাঠায়া দিছি...', 'এক সাথে কয়টা একাউন্ট লগইন করে রাখা যায় ব্রো?'], 'rm': ['eta kono post holo mia abal', 'Delete koren post…', 'ji bai osadaron hoice but full detailes janty parlam na, r eta ki bd te sob jaygatei powa jabe?', 'vai…ar 24 hour wait koren..amar fb id back ashbe ami request pathaia disi…', 'Ak sathe koyta account login kre rakha jay bro?']}


In [39]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load dataset
dataset_dict = load_dataset("SKNahin/bengali-transliteration-data")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50")

# Filter invalid rows (None or empty)
dataset_dict = dataset_dict.filter(lambda example: example['rm'] is not None and example['bn'] is not None)
dataset_dict = dataset_dict.filter(lambda example: len(example['rm'].strip()) > 0 and len(example['bn'].strip()) > 0)

Filter:   0%|          | 0/5006 [00:00<?, ? examples/s]

In [40]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['bn', 'rm'],
        num_rows: 5006
    })
})

In [41]:
def preprocess_function(examples):
    inputs = examples['rm']  # Banglish text
    targets = examples['bn']  # Bengali text

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    # Tokenize targets
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    # Add labels to the model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [42]:
# Apply preprocessing
tokenized_dataset = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/5006 [00:00<?, ? examples/s]

In [43]:
# Inspect the tokenized dataset
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['bn', 'rm', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5006
    })
})


In [44]:
print(dataset_dict['train'][0:5])


{'bn': ['স্ক্রোল করে ২০/৩০ সেকেন্ড এর ভিডিও পান নাই???', 'ও গুলা টরেন্ট সাইট এ পাবেন', 'ভক্কর চক্কর পোস্ট একটা করলেই এপ্রুভড.… নিশ্চই  ঘাবলা আছে', 'আমি টেস্ট করেই কোড দিছি…', 'এতো কষ্টের কি আছে সাকিবওয়াপ.টক,সাকিবওয়াপ.মল&এআইওভিডিওড্ল.মল থেকে সহজেই ডাউনলোড করা যায়'], 'rm': ['scroll kore 20/30 second er video pann nai???', 'o gula Torrent site e paben', 'vokkor chokkor post akta korlei approved…. nishchoi ghabla ache', 'ami test koreii code disi…', 'eto koster ki ache shakibwap.tk,shakibwap.ml&aiovideodl.ml theke shohojei downlod kora jay']}


In [45]:
print(tokenized_dataset["train"][0:5])


{'bn': ['স্ক্রোল করে ২০/৩০ সেকেন্ড এর ভিডিও পান নাই???', 'ও গুলা টরেন্ট সাইট এ পাবেন', 'ভক্কর চক্কর পোস্ট একটা করলেই এপ্রুভড.… নিশ্চই  ঘাবলা আছে', 'আমি টেস্ট করেই কোড দিছি…', 'এতো কষ্টের কি আছে সাকিবওয়াপ.টক,সাকিবওয়াপ.মল&এআইওভিডিওড্ল.মল থেকে সহজেই ডাউনলোড করা যায়'], 'rm': ['scroll kore 20/30 second er video pann nai???', 'o gula Torrent site e paben', 'vokkor chokkor post akta korlei approved…. nishchoi ghabla ache', 'ami test koreii code disi…', 'eto koster ki ache shakibwap.tk,shakibwap.ml&aiovideodl.ml theke shohojei downlod kora jay'], 'input_ids': [[250004, 192046, 20867, 387, 108355, 17932, 72, 1202, 2652, 19, 24, 14, 7273, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [250004, 36, 3788

In [46]:
from transformers import MBartForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments


In [47]:
# Load the model
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")


pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [48]:
# Set up the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",                # Output directory for model checkpoints
    evaluation_strategy="epoch",          # Evaluate at the end of every epoch
    learning_rate=5e-5,                   # Learning rate
    per_device_train_batch_size=8,        # Batch size for training
    per_device_eval_batch_size=8,         # Batch size for evaluation
    weight_decay=0.01,                    # Weight decay for optimization
    save_total_limit=2,                   # Limit the number of saved checkpoints
    num_train_epochs=3,                   # Number of training epochs
    predict_with_generate=True,           # Use `generate` for evaluation
    logging_dir="./logs",                 # Directory for logging
    logging_steps=200,                    # Log every 200 steps
    save_strategy="epoch",                # Save model checkpoint at the end of each epoch
    push_to_hub=False                     # Set to True if you want to push to Hugging Face Hub
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [49]:
# Define the data collator
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [51]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00


In [53]:
pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 6.8 MB/s eta 0:00:00


In [54]:
import evaluate

# Define metrics for evaluation
metric = evaluate.load("sacrebleu")


In [55]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Remove padding
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    # Compute BLEU score
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [59]:
# Split the dataset into training (80%) and validation (20%)
train_test_split = dataset_dict["train"].train_test_split(test_size=0.2, seed=42)

In [66]:
from datasets import DatasetDict
# Create a new DatasetDict with train and validation splits
tokenized_dataset = DatasetDict({
    "train": train_test_split["train"],
    "validation": train_test_split["test"]
})

KeyError: 'bn'

In [61]:
print("Train dataset size:", len(tokenized_dataset["train"]))
print("Validation dataset size:", len(tokenized_dataset["validation"]))


Train dataset size: 4004
Validation dataset size: 1002


In [62]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


<ipython-input-62-0966341e4de7>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [64]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    remove_unused_columns=False,  # Add this line
    logging_dir="./logs",
    logging_steps=100
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [67]:
# Remove unused columns (only keep the ones the model needs)
# train_dataset = train.remove_columns(["bn", "rm"])
train_dataset = tokenized_dataset["train"].remove_columns(["bn", "rm"])
# eval_dataset = validation.remove_columns(["bn", "rm"])
eval_dataset = tokenized_dataset["validation"].remove_columns(["bn", "rm"])


In [68]:
# Initialize the trainer with the updated datasets
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

<ipython-input-68-9c06bce35cae>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
# Train the model
trainer.train()

In [ ]:
# Example inputs for inference
example_inputs = ["ami bangladesh theke bolchi", "valo lage bangla bolte"]

# Tokenize the inputs
inputs = tokenizer(example_inputs, return_tensors="pt", max_length=128, truncation=True, padding="max_length")

# Generate predictions
outputs = model.generate(inputs["input_ids"], max_length=128, num_beams=4)

# Decode the predictions
decoded_predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print("Translated Outputs:", decoded_predictions)
